In [1]:
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.common.exceptions import TimeoutException
# xpath = '/html/body/div[2]/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div/' +\
#     'div[3]/div/table/tbody/tr[3]/td[6]/div/div'
# driver.get("https://www.surf-forecast.com/breaks/Trial-Harbour/forecasts/latest/six_day")

# element = driver.find_element(By.XPATH, sel_class)

In [224]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import math

In [65]:
driver = webdriver.Chrome()
driver.get("https://www.surf-forecast.com/breaks/Trial-Harbour/forecasts/latest")

rating = driver.find_element(By.CLASS_NAME, 'forecast-table-rating')
time = driver.find_element(By.CLASS_NAME, 'forecast-table-time')
table = driver.find_element(By.CLASS_NAME, 'forecast-table__basic')


In [123]:
df.to_csv('~/Desktop/break_urls.csv')

In [154]:
pages_df

,break_name,url
0,https://www.surf-forecast.com/breaks/Tenth-Str...,10th Street (Avalon)
1,https://www.surf-forecast.com/breaks/Agate-and...,Agate and Pearl Street
2,https://www.surf-forecast.com/breaks/Anakena,Anakena
3,https://www.surf-forecast.com/breaks/Onehundre...,120th Street
4,https://www.surf-forecast.com/breaks/Agate-Beach,Agate Beach
...,...,...
8095,https://www.surf-forecast.com/breaks/Anaehooma...,Anaehoomalu Bay_A-Bay
8096,https://www.surf-forecast.com/breaks/Aracaju,Aracaju
8097,https://www.surf-forecast.com/breaks/Aganoa-Beach,Aganoa Beach
8098,https://www.surf-forecast.com/breaks/Anael,Anael


In [196]:
pages_df['page_number']

0       <selenium.webdriver.remote.webelement.WebEleme...
1       <selenium.webdriver.remote.webelement.WebEleme...
2       <selenium.webdriver.remote.webelement.WebEleme...
3       <selenium.webdriver.remote.webelement.WebEleme...
4       <selenium.webdriver.remote.webelement.WebEleme...
                              ...                        
2095    <selenium.webdriver.remote.webelement.WebEleme...
2096    <selenium.webdriver.remote.webelement.WebEleme...
2097    <selenium.webdriver.remote.webelement.WebEleme...
2098    <selenium.webdriver.remote.webelement.WebEleme...
2099    <selenium.webdriver.remote.webelement.WebEleme...
Name: page_number, Length: 2100, dtype: object

In [222]:
if len(pages_df) < 3000:
    max_page = 0
    pages_df = pd.DataFrame({'break_name': [], 'url': [], 'page_number': []})
    
if len(pages_df) == 0:
    df = pd.DataFrame({'break_name': [], 'break_url': [], 'time': [], 'rating': []})

driver = webdriver.Chrome()
driver.set_window_position(2000,2000)
driver.get('https://www.surf-forecast.com/breaks/')
breaks = driver.find_elements(By.CLASS_NAME, 'list_table')

for i in driver.find_element(By.CLASS_NAME, 'pagination').find_elements(By.TAG_NAME, 'a'):
    try:
        cur_page = int(i.text)
        if cur_page > max_page:
            max_page = cur_page
    except:
        continue
        
if max_page > pages_df['page_number'].max() or math.isnan(pages_df['page_number'].max()):
    for page_num in range(1, max_page + 1):
        driver.get('https://www.surf-forecast.com/breaks?page={}'.format(page_num))
        driver.find_elements(By.CLASS_NAME, 'pagination') 
        breaks = driver.find_elements(By.CLASS_NAME, 'list_table')

        for surf_spot in breaks[0].find_elements(By.TAG_NAME, 'a'):
            pages_df = pd.concat([pages_df, pd.DataFrame({
                'break_name': [surf_spot.text]
                , 'url': [surf_spot.get_attribute('href')]
                , 'page_number': [page_num]
            })], ignore_index=True)
        
for index, row in pages_df.iterrows():
    # If this were in production Remove code until HERE and timestamp each pull
    if (df['break_name'] == row['break_name']).sum() > 40:
        continue
    print(row['break_name'])
    # HERE
    
    driver.get(row['url'] + '/forecasts/latest')

    ratings = driver.find_elements(By.CLASS_NAME, 'star-rating__rating')
    times = driver.find_elements(By.CLASS_NAME, 'forecast-table-time__cell')

    for time, rating in zip(times, ratings):
        df = pd.concat([df, pd.DataFrame({
            'break_name': [row['break_name']]
            , 'break_url': [row['url']]
            , 'time': [time.text]
            , 'rating': [rating.text]
        })], ignore_index=True)

KeyboardInterrupt: 

In [221]:
pages_df

,break_name,url,page_number
